In [12]:
import math as m
import numpy as np
from numpy import linalg as LA
import matplotlib.pyplot as plt
import time
import matplotlib as ml
import matplotlib.pyplot as pp
%matplotlib inline

In [80]:
n = 100
h = 1.0 / n
eps = 0.01

In [81]:
def showsol(sol, n):
    pp.imshow(sol.reshape((n, n)),cmap=ml.cm.Blues,interpolation='none',origin='lower')
    
def SetF(f, n):
    x = np.linspace(0.0, 1.0, num=n, endpoint=False)
    y = np.linspace(0.0, 1.0, num=n, endpoint=False)
    for i in range(n):
        for j in range(n):
            f[i * n + j] = m.cos(m.pi * x[i] * 10) * m.cos(m.pi * y[j] * 1) / n**2 
            
def LUA(x, n, i, j):
    return 2 * x[0] - x[n] - x[1]

def RUA(x, n, i, j):
    return 2 * x[n - 1] - x[n + n - 1] - x[n - 2]

def LLA(x, n, i, j):
    return 2 * x[(n - 1) * n] - x[(n - 2) * n] - x[(n - 1) * n + 1]

def RLA(x, n, i, j):
    return 2 * x[(n - 1) * n + n - 1] - x[(n - 2) * n + n - 1] - x[(n - 1) * n + n - 2]

def Left(x, n, i, j):
    return 3 * x[i * n + j] - x[i * n + j + 1] - x[(i - 1) * n + j] - x[(i + 1) * n + j]

def Right(x, n, i, j):
    return 3 * x[i * n + j] - x[i * n + j - 1] - x[(i - 1) * n + j] - x[(i + 1) * n + j]

def Low(x, n, i, j):
    return 3 * x[i * n + j] - x[(i - 1) * n + j] - x[i* n + j - 1] - x[i * n + j + 1]

def Up(x, n, i, j):
    return 3 * x[i * n + j] - x[(i + 1) * n + j] - x[i* n + j - 1] - x[i * n + j + 1]

def Inside(x, n, i, j):
    return 4 * x[i * n + j] - x[(i + 1) * n + j] - x[i* n + j - 1] - x[i * n + j + 1] - x[(i - 1) * n + j]

def SetFuncVec(f, n):
    for i in range(n):
        for j in range(n):
            if i == 0 and j == 0:
                f.append(LUA)
                continue
            elif i == 0 and j in list(range(1, n - 1)):
                f.append(Up)
                continue
            elif i == 0 and j == n - 1:
                f.append(RUA)
                continue
            elif i == n - 1 and j == 0:
                f.append(LLA)
                continue
            elif i == n - 1 and j in list(range(1, n - 1)):
                f.append(Low)
                continue
            elif i == n - 1 and j == n - 1:
                f.append(RLA)
                continue
            elif i in list(range(1, n - 1)) and j == 0:
                f.append(Left)
                continue
            elif i in list(range(1, n - 1)) and j == n - 1:
                f.append(Right)
                continue
            else:
                f.append(Inside)
                continue
                
def Operator(f, x, n):
    res = np.zeros( n**2 )
    
    for i in range(n):
        for j in range(n):
            res[i * n + j] = f[i * n + j](x, n, i, j)
            
    return res

def rowA(i, n):
    A = {}
    for j in [i - n, i - 1, i, i + 1, i + n]:
        if j >= n**2 or j < 0:
            continue
        if i == j and ( i % n == 0 or i % n == n - 1) and i != 0 and i != n - 1 and i != n**2 - n and i != n**2 - 1 and (i not in list(range(n**2 - n + 1, n**2 - 1))): 
            A[j] = 3.0 
            continue
        if i == j and( i in list(range(1, n - 1)) or i in list(range(n**2 - n + 1, n**2 - 1))):
            A[j] = 3.0
            continue
        elif i == j and ( i == 0 or i == n - 1 or i == n**2 - n or i == n**2 - 1):
            A[j] = 2.0  
            continue
        elif (j == i - 1 and i % n != 0) or (j == i + 1 and i % n != n - 1): 
            A[j] = -1.0 
            continue
        elif (j == i - n or j == i + n): 
            A[j] = -1.0 
            continue
        elif i == j:
            A[j] = 4.0  
            continue
    return A

def Matr(n):
    A = np.zeros((n**2, n**2))
    for i in range(n**2):
        for j in range(n**2):
            if i == j and ( i % n == 0 or i % n == n - 1) and i != 0 and i != n - 1 and i != n**2 - n and i != n**2 - 1 and (i not in list(range(n**2 - n + 1, n**2 - 1))): 
#                 print(i, ' ', j)
                A[i][j] = 3.0 
                continue
            if i == j and( i in list(range(1, n - 1)) or i in list(range(n**2 - n + 1, n**2 - 1))):
                A[i][j] = 3.0
                continue
            elif i == j and ( i == 0 or i == n - 1 or i == n**2 - n or i == n**2 - 1):
                A[i][j] = 2.0  
                continue
            elif (j == i - 1 and i % n != 0) or (j == i + 1 and i % n != n - 1): 
                A[i][j] = -1.0 
                continue
            elif (j == i - n or j == i + n): 
                A[i][j] = -1.0 
                continue
            elif i == j:
                A[i][j] = 4.0  
                continue
#     for i in range(n**2):
#         for j in range(n**2):
#             print(A[i][j], end = ' ')
#         print()
    return A

# Matr(n)

In [82]:
f = np.zeros( n**2 )
SetF(f, n)

func = []
SetFuncVec(func, n)

In [83]:
M = 8 * n**2 * ((m.sin((m.pi * h) / 2.0))**2)
mu = 8 * n**2 * ((m.cos((m.pi * h) / 2.0))**2)

print(M)
print(mu)

t1 = (M + mu) / 2.0 - ((M - mu) / 2.0) * m.cos(m.pi / (4.0))
t2 = (M + mu) / 2.0 - ((M - mu) / 2.0) * m.cos((m.pi * 3.0) / (4.0))
# print(t1)
# print(t2)
t1 = 1.0 / t1
t2 = 1.0 / t2

tau1 = t1
tau2 = t2

print(t1)
print(t2)

19.737585370737715
79980.26241462927
1.4647654765876496e-05
8.525377890460506e-05


In [76]:
x = np.ones( n**2 )
for i in range(n**2):
    x[i] = i * 0.1 + x[i]
x_next = np.zeros( n**2 )

step = 0

start_time = time.time()
error = LA.norm(Operator(func, x, n) - f, axis=0)

print("Error outside>> ", error)

# eps = 0.000000001

while(error > eps):
    if(step % 2 == 0):
        tau = tau1
    else:
        tau = tau2
#     print('tau = ', tau)
#     print(Operator(func, x, n))
#     print(tau * f)
#     for i in range(n**2):
#         print(i, ": ", x[i], " + ", tau * f[i], " = ", x[i] + tau * f[i])
    x_next =  np.copy(x - tau * Operator(func, x, n)  + tau * f)
    error = LA.norm(Operator(func, x, n) - f, axis=0)
    print("Error >> ", error)
#     print(x_next)
#     print(x_next[0])
    x = np.copy(x_next)
    print('iter = ', step)
    step += 1

print("--- %s seconds ---" % (time.time() - start_time))
print("|Ax - f| = ", LA.norm(Operator(func, x, n) - f, axis=0))
print('iter = ', step)
pp.figure(figsize=(12,12))
showsol(x, n)

Error outside>>  12.665331618240401
Error >>  12.665331618240401
iter =  0
Error >>  12.660671994341387
iter =  1
Error >>  12.634483494256026
iter =  2
Error >>  12.629857910448939
iter =  3
Error >>  12.603860678184821
iter =  4
Error >>  12.599268830314255
iter =  5
Error >>  12.573461156374593
iter =  6
Error >>  12.568902743053586
iter =  7
Error >>  12.543282933512856
iter =  8
Error >>  12.538757656100085
iter =  9
Error >>  12.513324032476348
iter =  10
Error >>  12.508831595053783
iter =  11
Error >>  12.48358249418165
iter =  12
Error >>  12.479122603532028
iter =  13
Error >>  12.454056377436777
iter =  14
Error >>  12.449628743021412
iter =  15
Error >>  12.42474375879359
iter =  16
Error >>  12.420348092730228
iter =  17
Error >>  12.395642732401127
iter =  18
Error >>  12.391278749442014
iter =  19
Error >>  12.366751409860006
iter =  20
Error >>  12.362418827370037
iter =  21
Error >>  12.338067920077538
iter =  22
Error >>  12.333766458012555
iter =  23
Error >>  12.309

Error >>  10.103841322046172
iter =  244
Error >>  10.101735170469626
iter =  245
Error >>  10.089893584173774
iter =  246
Error >>  10.087799248461447
iter =  247
Error >>  10.076024052607599
iter =  248
Error >>  10.07394143661376
iter =  249
Error >>  10.06223209110262
iter =  250
Error >>  10.060161099589683
iter =  251
Error >>  10.048517069424289
iter =  252
Error >>  10.046457608053913
iter =  253
Error >>  10.03487836328966
iter =  254
Error >>  10.032830338613943
iter =  255
Error >>  10.02131535430899
iter =  256
Error >>  10.019278673761782
iter =  257
Error >>  10.007827429928023
iter =  258
Error >>  10.005802001816212
iter =  259
Error >>  9.994413983370677
iter =  260
Error >>  9.992399716865629
iter =  261
Error >>  9.98107441358224
iter =  262
Error >>  9.979071218711322
iter =  263
Error >>  9.967808125173192
iter =  264
Error >>  9.965815912811431
iter =  265
Error >>  9.954614528363564
iter =  266
Error >>  9.952633210225187
iter =  267
Error >>  9.941493038927666
i

Error >>  8.841913850471638
iter =  486
Error >>  8.840739448090037
iter =  487
Error >>  8.83413379704524
iter =  488
Error >>  8.832964192358068
iter =  489
Error >>  8.826385507439484
iter =  490
Error >>  8.825220668511546
iter =  491
Error >>  8.818668770310603
iter =  492
Error >>  8.817508665479425
iter =  493
Error >>  8.810983376118658
iter =  494
Error >>  8.809827973991634
iter =  495
Error >>  8.803329117109723
iter =  496
Error >>  8.802178386561605
iter =  497
Error >>  8.795705787298633
iter =  498
Error >>  8.794559697468845
iter =  499
Error >>  8.788113182451632
iter =  500
Error >>  8.786971702741761
iter =  501
Error >>  8.780551100069404
iter =  502
Error >>  8.779414200140621
iter =  503
Error >>  8.773019339370158
iter =  504
Error >>  8.771886989140661
iter =  505
Error >>  8.765517701272941
iter =  506
Error >>  8.764389870915519
iter =  507
Error >>  8.758045988381069
iter =  508
Error >>  8.756922648320591
iter =  509
Error >>  8.75060400496584
iter =  510
Er

Error >>  8.09288016257497
iter =  724
Error >>  8.092110175249537
iter =  725
Error >>  8.087777976397533
iter =  726
Error >>  8.087010347303792
iter =  727
Error >>  8.082691408173195
iter =  728
Error >>  8.081926124854991
iter =  729
Error >>  8.077620375445802
iter =  730
Error >>  8.076857425536641
iter =  731
Error >>  8.072564796352626
iter =  732
Error >>  8.07180416757497
iter =  733
Error >>  8.067524589619204
iter =  734
Error >>  8.066766269783576
iter =  735
Error >>  8.062499674554115
iter =  736
Error >>  8.061743651558432
iter =  737
Error >>  8.057489971044022
iter =  738
Error >>  8.05673623287271
iter =  739
Error >>  8.052495399548445
iter =  740
Error >>  8.05174393427183
iter =  741
Error >>  8.04751588109497
iter =  742
Error >>  8.046766676868405
iter =  743
Error >>  8.042551337274173
iter =  744
Error >>  8.041804382337416
iter =  745
Error >>  8.037601690234805
iter =  746
Error >>  8.03685697291118
iter =  747
Error >>  8.032666862678896
iter =  748
Error 

iter =  950
Error >>  7.598495801393205
iter =  951
Error >>  7.595313686038301
iter =  952
Error >>  7.594749587654491
iter =  953
Error >>  7.591575204729332
iter =  954
Error >>  7.5910124752690695
iter =  955
Error >>  7.587845791632167
iter =  956
Error >>  7.587284425235495
iter =  957
Error >>  7.584125407944748
iter =  958
Error >>  7.583565398786941
iter =  959
Error >>  7.580414015098228
iter =  960
Error >>  7.579855357389498
iter =  961
Error >>  7.576711574755115
iter =  962
Error >>  7.576154262740392
iter =  963
Error >>  7.5730180488075955
iter =  964
Error >>  7.572462076766216
iter =  965
Error >>  7.569333399375766
iter =  966
Error >>  7.568778761621232
iter =  967
Error >>  7.565657588805873
iter =  968
Error >>  7.565104279685639
iter =  969
Error >>  7.561990579668741
iter =  970
Error >>  7.561438593563912
iter =  971
Error >>  7.558332334757925
iter =  972
Error >>  7.557781666082993
iter =  973
Error >>  7.554682817088187
iter =  974
Error >>  7.55413346029080

...

iter =  47755
Error >>  1.3448075957078496
iter =  47756
Error >>  1.3447954101196897
iter =  47757
Error >>  1.3447267897411586
iter =  47758
Error >>  1.3447146048852077
iter =  47759
Error >>  1.3446459886298934
iter =  47760
Error >>  1.3446338045060988
iter =  47761
Error >>  1.3445651923737607
iter =  47762
Error >>  1.3445530089820725
iter =  47763
Error >>  1.3444844009724388
iter =  47764
Error >>  1.3444722183128317
iter =  47765
Error >>  1.3444036144256843
iter =  47766
Error >>  1.3443914324980977
iter =  47767
Error >>  1.3443228327331842
iter =  47768
Error >>  1.344310651537569
iter =  47769
Error >>  1.3442420558946284
iter =  47770
Error >>  1.3442298754309467
iter =  47771
Error >>  1.3441612839097519
iter =  47772
Error >>  1.3441491041779725
iter =  47773
Error >>  1.3440805167782428
iter =  47774
Error >>  1.3440683377783085
iter =  47775
Error >>  1.3439997544998303
iter =  47776
Error >>  1.3439875762316913
iter =  47777
Error >>  1.3439189970742096
iter =  4777

iter =  48148
Error >>  1.329050010886947
iter =  48149
Error >>  1.328982193943125
iter =  48150
Error >>  1.3289701517523014
iter =  48151
Error >>  1.3289023388834247
iter =  48152
Error >>  1.3288902974161918
iter =  48153
Error >>  1.3288224886219993
iter =  48154
Error >>  1.3288104478783
iter =  48155
Error >>  1.328742643158575
iter =  48156
Error >>  1.3287306031383816
iter =  48157
Error >>  1.328662802492858
iter =  48158
Error >>  1.3286507631961129
iter =  48159
Error >>  1.3285829666245497
iter =  48160
Error >>  1.3285709280512357
iter =  48161
Error >>  1.3285031355533914
iter =  48162
Error >>  1.328491097703425
iter =  48163
Error >>  1.3284233092790338
iter =  48164
Error >>  1.328411272152408
iter =  48165
Error >>  1.3283434878012548
iter =  48166
Error >>  1.3283314513978999
iter =  48167
Error >>  1.3282636711197415
iter =  48168
Error >>  1.328251635439613
iter =  48169
Error >>  1.328183859234192
iter =  48170
Error >>  1.3281718242772673
iter =  48171
Error >>

Error >>  1.3201475119762633
iter =  48372
Error >>  1.3201355498383875
iter =  48373
Error >>  1.320068187769496
iter =  48374
Error >>  1.320056226350388
iter =  48375
Error >>  1.3199888683290908
iter =  48376
Error >>  1.319976907628723
iter =  48377
Error >>  1.319909553654789
iter =  48378
Error >>  1.3198975936731265
iter =  48379
Error >>  1.3198302437463079
iter =  48380
Error >>  1.3198182844832693
iter =  48381
Error >>  1.3197509386033552
iter =  48382
Error >>  1.3197389800589097
iter =  48383
Error >>  1.3196716382256395
iter =  48384
Error >>  1.3196596803997593
iter =  48385
Error >>  1.3195923426128664
iter =  48386
Error >>  1.3195803855054973
iter =  48387
Error >>  1.3195130517647435
iter =  48388
Error >>  1.319501095375865
iter =  48389
Error >>  1.319433765681038
iter =  48390
Error >>  1.3194218100105737
iter =  48391
Error >>  1.3193544843613971
iter =  48392
Error >>  1.3193425294093224
iter =  48393
Error >>  1.319275207805597
iter =  48394
Error >>  1.319263

Error >>  1.3117656158180826
iter =  48584
Error >>  1.311753729630358
iter =  48585
Error >>  1.3116867952575173
iter =  48586
Error >>  1.3116749097839915
iter =  48587
Error >>  1.3116079794330657
iter =  48588
Error >>  1.311596094673711
iter =  48589
Error >>  1.3115291683444508
iter =  48590
Error >>  1.3115172842992258
iter =  48591
Error >>  1.3114503619913835
iter =  48592
Error >>  1.3114384786602429
iter =  48593
Error >>  1.3113715603735954
iter =  48594
Error >>  1.3113596777564813
iter =  48595
Error >>  1.3112927634907836
iter =  48596
Error >>  1.3112808815876664
iter =  48597
Error >>  1.3112139713426711
iter =  48598
Error >>  1.3112020901535186
iter =  48599
Error >>  1.311135183928974
iter =  48600
Error >>  1.3111233034537306
iter =  48601
Error >>  1.3110564012494157
iter =  48602
Error >>  1.311044521488024
iter =  48603
Error >>  1.3109776233036847
iter =  48604
Error >>  1.3109657442561227
iter =  48605
Error >>  1.310898850091522
iter =  48606
Error >>  1.3108

KeyboardInterrupt: 

In [ ]:
aii = 0
cij = 0
d = 0
eps = 0.001

step = 0

start_time = time.time()
iterId = 0

x = np.ones( n**2 )
for i in range(n**2):
    x[i] = i * 0.1 * x[i]
x_prev = np.zeros( n**2 )

error = LA.norm(Operator(func, x, n) - f, axis=0)

while(error > eps):
    x_prev = np.copy(x)
    for i in range(n**2):
        val = 0.0
        A = rowA(i, n)
        aii = A[i]
        for j in [i - n, i - 1, i + 1, i + n]:
            try:
                aij = A[j]
                val += aij * x[j]
            except:
                None
        x[i] = (f[i] - val) / aii
        
    error = LA.norm(Operator(func, x, n) - f, axis=0)
    print("Error >> ", error)
    step += 1
    
print("--- %s seconds ---" % (time.time() - start_time))
print("|Ax - f| = ", LA.norm(Operator(func, x, n) - f))
print('iter = ', step)
pp.figure(figsize=(12,12))
showsol(x, n)

Error >>  100.24772952321572
Error >>  86.73576584981413
Error >>  78.99250210405779
Error >>  73.74123159909429
Error >>  69.8534418776188
Error >>  66.80939952955093
Error >>  64.33131045302493
Error >>  62.255506185964165
Error >>  60.478442940050016
Error >>  58.93092569195487
Error >>  57.56466043365187
Error >>  56.344732488770134
Error >>  55.24515519028743
Error >>  54.24610960586715
Error >>  53.33216432798829
Error >>  52.491088310144015
Error >>  51.71303606976185
Error >>  50.989974294110574
Error >>  50.315269378829974
Error >>  49.68338492383878
Error >>  49.089656013933705
Error >>  48.53011817299487
Error >>  48.001375934186136
Error >>  47.5005005719751
Error >>  47.0249496099078
Error >>  46.57250280221143
Error >>  46.14121072774302
Error >>  45.72935314627668
Error >>  45.33540498779603
Error >>  44.95800836583105
Error >>  44.59594938633384
Error >>  44.248138804935934
Error >>  43.913595795734814
Error >>  43.59143425351377
Error >>  43.28085117226857
Error >>  42

In [ ]:
# x = 3 * np.ones( n**2 )

# for i in range(n**2):
#     x[i] = i * 0.1 + x[i]

# A = Matr(n)
# np.dot(A, x)

In [ ]:
# Operator(func, x, n)